In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    
@dataclass(frozen=True)

class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from ASL_alphabet.constants import *
from ASL_alphabet.utils import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "ASL_Dataset","Train")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
    
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        #passing all datagenerator_kwargs  if augmentation is False
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs     #** for keyword argument
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,  #fetch the data
            subset="validation",   #validation subset for spliting dataset for validation
            shuffle=False,         #since it's a image data we can either shuffle with Boolean
            **dataflow_kwargs    #passing keyword argument
        )
        #if augmentation is True then
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                featurewise_center=False,
                samplewise_center=False, 
                featurewise_std_normalization=False,
                samplewise_std_normalization=False,
                zca_whitening=False,
                rotation_range=10,
                zoom_range = 0.1, 
                width_shift_range=0.1,
                height_shift_range=0.1,
                horizontal_flip=False,
                vertical_flip=False,
                **datagenerator_kwargs
            )
        else:
            #remain same
            train_datagenerator = valid_datagenerator


        #specify the training data directory
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True, #suffle is to be done  while training
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        #we need steps per epochs and validation steps
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        #call the model and fit it
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

2023-07-11 19:38:33.648324: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 19:38:34.458379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2023-07-11 19:38:35,413: INFO: utils]: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2023-07-11 19:38:35,414: INFO: utils]: NumExpr defaulting to 8 threads.


In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    # @property
    # def _create_tb_callbacks(self):
    #     timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
    #     tb_running_log_dir = os.path.join(
    #         self.config.tensorboard_root_log_dir,
    #         f"tb_logs_at_{timestamp}",
    #     )
    #     return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    # @property
    # def _create_ckpt_callbacks(self):
    #     return tf.keras.callbacks.ModelCheckpoint(
    #         filepath=self.config.checkpoint_model_filepath,
    #         save_best_only=True
    #     )
    @property
    def _create_tensorboard_callback(self):
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,time.strftime("%Y%m%d-%H%M%S"))
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
        return tensorboard_callback
    

    @property
    def _create_ckpt_callbacks(self):
        checkpoint_model_filepath = str(self.config.checkpoint_model_filepath)  # Convert to string if it's a PosixPath object
        
        return tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_model_filepath, 
                save_best_only=True
            )
         
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tensorboard_callback,
            self._create_ckpt_callbacks
        ]

In [7]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-07-11 19:38:35,837: INFO: utility]: yaml file: configs/config.yaml loaded successfully
[2023-07-11 19:38:35,839: INFO: utility]: yaml file: params.yaml loaded successfully
[2023-07-11 19:38:35,840: INFO: utility]: created directory at: artifacts
[2023-07-11 19:38:35,841: INFO: utility]: created directory at: artifacts/prepare_callbacks/checkpoint_dir
[2023-07-11 19:38:35,841: INFO: utility]: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
[2023-07-11 19:38:35,843: INFO: utility]: created directory at: artifacts/training


2023-07-11 19:38:37.076760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-11 19:38:37.170579: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-11 19:38:37.170743: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 33128 images belonging to 28 classes.
Found 132542 images belonging to 28 classes.
Epoch 1/20


2023-07-11 19:38:41.677436: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-07-11 19:38:43.158428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-07-11 19:38:44.255741: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-11 19:38:44.266917: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ef941fb0a40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-11 19:38:44.266969: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-07-11 19:38:44.288769: I tensorflow/compiler/mlir/tensorflow/utils/dump_

1035/1035 [==============================] - 223s 211ms/step - loss: 0.1745 - accuracy: 0.9458 - val_loss: 0.0834 - val_accuracy: 0.9768
Epoch 2/20


/home/lang_chain/Documents/ASL_alphabet/.venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1035/1035 [==============================] - 192s 185ms/step - loss: 0.0149 - accuracy: 0.9951 - val_loss: 2.8287 - val_accuracy: 0.5548
Epoch 3/20
1035/1035 [==============================] - 186s 179ms/step - loss: 0.0118 - accuracy: 0.9961 - val_loss: 3.6011 - val_accuracy: 0.4891
Epoch 4/20
1035/1035 [==============================] - 193s 187ms/step - loss: 0.0108 - accuracy: 0.9965 - val_loss: 4.6637 - val_accuracy: 0.5563
Epoch 5/20
1035/1035 [==============================] - 197s 190ms/step - loss: 0.0088 - accuracy: 0.9973 - val_loss: 11.8214 - val_accuracy: 0.3709
Epoch 6/20
1035/1035 [==============================] - 192s 185ms/step - loss: 0.0089 - accuracy: 0.9975 - val_loss: 0.0063 - val_accuracy: 0.9978
Epoch 7/20
1035/1035 [==============================] - 184s 178ms/step - loss: 0.0071 - accuracy: 0.9980 - val_loss: 9.9502 - val_accuracy: 0.4358
Epoch 8/20
1035/1035 [==============================] - 185s 179ms/step - loss: 0.0072 - accuracy: 0.9980 - val_loss: 0.07

In [8]:

os.path.join("artifacts/data_ingestion", "ASL_Dataset","Train")

'artifacts/data_ingestion/ASL_Dataset/Train'